In [2]:
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier

from ssri_interactions.io import load_derived_generic
from ssri_interactions.config import ExperimentInfo, Config
from ssri_interactions.decoding.loaders import FSDecodeDataLoader, FSFastDecodeDataLoader
from ssri_interactions.decoding.preprocessors import DecodePreprocessor
from ssri_interactions.decoding.runners import DecodeRunner
from ssri_interactions.decoding.decoders import Decoder
from ssri_interactions.decoding.shuffle import shuffle_X

# %load_ext autoreload
# %autoreload 2

c:\ProgramData\Miniconda3\envs\ssri\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


# Slow TS

In [6]:
def make_slow_ts_runner(loader):
    preprocessor = DecodePreprocessor(
        thresh_empty=2,
    )
    spikes, states = loader()
    spikes, states = preprocessor(spikes, states)
    estimator = LGBMClassifier(n_estimators=50, max_depth=8, num_leaves=30)
    cv = KFold(n_splits=5, shuffle=True)
    decoder = Decoder(estimator=estimator, cv=cv, shuffler=shuffle_X)
    runner = DecodeRunner(
        loader=loader,
        preprocessor=preprocessor,
        decoder=decoder,
        nboot=75,
    )
    return runner

#### Loaders

In [10]:
neuron_types = load_derived_generic("neuron_types.csv").query("group in ('CIT', 'SAL')")
sessions = neuron_types.query("experiment_name == 'HAMILTON'").session_name.unique().tolist()
session = sessions[0]
loader_post_shock = FSDecodeDataLoader(session_name=session)
loader_shock_only = FSDecodeDataLoader(session_name=session, t_stop=600)


#### Shock Only

In [11]:
runner_shock_only = make_slow_ts_runner(loader_shock_only)

pop, unit = runner_shock_only.run_multiple(sessions)
limit = runner_shock_only.run_multiple_limit(sessions=sessions, n_min=1, n_max=15)
dropout = runner_shock_only.run_multiple_dropout(sessions=sessions, neuron_types=("SR", "SIR", "FF"))


dd = Config.derived_data_dir / "decoding"
dd.mkdir(exist_ok=True)
pop.to_csv(dd / "fs_slow - shock only - pop.csv", index=False)
unit.to_csv(dd / "fs_slow - shock only - unit.csv", index=False)
limit.to_csv(dd / "fs_slow - shock only - limit.csv", index=False)
dropout.to_csv(dd / "fs_slow - shock only - dropout.csv", index=False)

100%|██████████| 75/75 [00:15<00:00,  4.70it/s]


#### Post-included

In [25]:
runner_post_shock = make_slow_ts_runner(loader_post_shock)

pop, unit = runner_post_shock.run_multiple(sessions)
limit = runner_post_shock.run_multiple_limit(sessions=sessions, n_min=1, n_max=15)
dropout = runner_post_shock.run_multiple_dropout(sessions=sessions, neuron_types=("SR", "SIR", "FF"))


dd = Config.derived_data_dir / "decoding"
dd.mkdir(exist_ok=True)
pop.to_csv(dd / "fs_slow - post-included - pop.csv", index=False)
unit.to_csv(dd / "fs_slow - post-included - unit.csv", index=False)
limit.to_csv(dd / "fs_slow - post-included - limit.csv", index=False)
dropout.to_csv(dd / "fs_slow - post-included - dropout.csv", index=False)

100%|██████████| 75/75 [01:11<00:00,  1.05it/s]


Cannot have number of splits n_splits=5 greater than the number of samples: n_samples=0.


#### Export

In [29]:
from sklearn.decomposition import PCA, KernelPCA
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVC


# sessions = ExperimentInfo.foot_shock_sessions_10min
# session = sessions[0]
loader = FSFastDecodeDataLoader(session_name=session, bin_width=0.1)
preprocessor = DecodePreprocessor(
    thresh_empty=1, 
)
spikes, states = loader()
spikes, states = preprocessor(spikes, states)

estimator = LGBMClassifier(n_estimators=40, max_depth=8, num_leaves=30)
estimator = make_pipeline(StandardScaler(), estimator)

cv = KFold(n_splits=5, shuffle=True)
decoder = Decoder(estimator=estimator, cv=cv, shuffler=shuffle_X)
runner = DecodeRunner(
    loader=loader,
    preprocessor=preprocessor,
    decoder=decoder,
    nboot=75,
)

In [30]:
pop, unit = runner.run_multiple(sessions)
limit = runner.run_multiple_limit(sessions=sessions, n_min=1, n_max=15)
dropout = runner.run_multiple_dropout(sessions=sessions, neuron_types=("SR", "SIR", "FF"))

100%|██████████| 75/75 [00:19<00:00,  3.79it/s]


Cannot have number of splits n_splits=5 greater than the number of samples: n_samples=1.


In [ ]:
pop.to_csv(dd / "fs_fast - pop.csv", index=False)
unit.to_csv(dd / "fs_fast - unit.csv", index=False)
limit.to_csv(dd / "fs_fast - limit.csv", index=False)
dropout.to_csv(dd / "fs_fast - dropout.csv", index=False)